<a href="https://colab.research.google.com/github/blade-git/Hotel-Review-Sentiment-Analysis/blob/main/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vaderSentiment

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**<라이브러리 임포트>**


*   pandas : 데이터를 불러오고 처리하기 위한 라이브러리.
*   CountVectorizer : 텍스트 데이터를 벡터화(문서-단어 행렬 생성)하기 위한 도구. LDA 모델 학습에 필요한 입력 데이터를 생성.
*   LatentDirichletAllocation : LDA 알고리즘을 구현한 클래스. 문서에서 숨겨진 주제를 추출하는 데 사용.
*   nltk & stopwords : 텍스트 전처리에 사용. 불용어(stopwords)를 제거하여 LDA 모델이 의미 있는 단어에 초점을 맞춤.








In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
from nltk.corpus import stopwords

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**<NLTK 불용어(stopwords) 리스트 다운로드>**


1.   nltk.download('stopwords') :


*   NLTK에서 제공하는 불용어(stopwords) 리스트를 다운로드.
*   불용어란 텍스트 분석에서 의미를 추가하지 않는 단어("the", "is", "in")들을 말함.


2.   stop_words = stopwords.words('english') :


*   NLTK가 제공하는 영어 불용어 리스트를 불러와서 stop_words 변수에 저장.
*   이 리스트에는 텍스트 분석에 불필요한 단어들이 포함되어 있으며, 주로 텍스트 전처리 과정에서 제거.





In [ ]:
# NLTK 불용어 리스트 다운로드
nltk.download('stopwords')
stop_words = stopwords.words('english')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/tripadvisor_hotel_reviews.csv')  # 파일 경로에 맞게 수정

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**<토픽모델링 수행 과정>**


1.   텍스트 전처리


*   Review 컬럼의 텍스트 데이터를 소문자로 변환하고, 특수문자 등을 제거하여 Cleaned_Review 컬럼에 저장.
*   데이터 정제를 통해 분석의 정확성을 높임.


2.   문서-단어 행렬 생성


*   CountVectorizer를 사용해 문서-단어 행렬을 생성.
*   불용어를 제거하고 상위 1,000개의 단어를 선택하여 문서-단어 행렬을 생성.


3.   LDA 모델 학습


*   LatentDirichletAllocation 클래스의 인스턴스를 생성하고, 문서-단어 행렬로 학습시켜 5개의 주제를 도출.
*   주제의 개수를 5개로 설정하고 결과 재현성을 보장하기 위해 난수 시드를 설정.
*   각 문서가 속하는 주제 확률 분포와 각 주제를 구성하는 단어들의 확률 분포를 학습.


4.   각 주제에 포함된 단어 출력


*   문서-단어 행렬의 단어 리스트를 반환하여 각 주제에서 확률이 높은 상위 10개의 단어를 추출.
*   각 주제를 대표하는 주요 단어들을 출력.











In [ ]:
# 1. 텍스트 전처리
# 리뷰 텍스트를 소문자로 변환하고, 불용어 제거 및 토큰화
data['Cleaned_Review'] = data['Review'].str.lower().replace('[^\w\s]', '', regex=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:3: DeprecationWarning: invalid escape sequence '\w'
<>:3: DeprecationWarning: invalid escape sequence '\w'
<ipython-input-28-aca5e5c0fa7b>:3: DeprecationWarning: invalid escape sequence '\w'
  data['Cleaned_Review'] = data['Review'].str.lower().replace('[^\w\s]', '', regex=True)


In [ ]:
# 2. 문서-단어 행렬 생성
# CountVectorizer를 사용해 텍스트 데이터를 벡터화
vectorizer = CountVectorizer(stop_words=stop_words, max_features=1000)
doc_term_matrix = vectorizer.fit_transform(data['Cleaned_Review'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 3. LDA 모델 학습
# 주제 수는 임의로 5로 설정 (필요 시 변경 가능)
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_model.fit(doc_term_matrix)

# 각 문서에 대한 토픽 예측
topic_predictions = lda_model.transform(doc_term_matrix)

# 데이터프레임에 토픽 열 추가
data['Topic'] = topic_predictions.argmax(axis=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 4. 각 주제에 포함된 단어 출력
words = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda_model.components_):
    print(f"Topic {topic_idx + 1}: ", [words[i] for i in topic.argsort()[-10:]])

Topic 1:  ['room', 'service', 'area', 'good', 'food', 'nice', 'great', 'beach', 'hotel', 'pool']
Topic 2:  ['like', 'staff', 'rooms', 'service', 'desk', 'night', 'stay', 'nt', 'hotel', 'room']
Topic 3:  ['rooms', 'helpful', 'friendly', 'stayed', 'room', 'location', 'stay', 'staff', 'great', 'hotel']
Topic 4:  ['rooms', 'clean', 'walk', 'nice', 'breakfast', 'great', 'location', 'good', 'room', 'hotel']
Topic 5:  ['room', 'day', 'people', 'time', 'great', 'good', 'food', 'nt', 'beach', 'resort']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# VADER 감성 분석 초기화
analyzer = SentimentIntensityAnalyzer()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**1. VADER 감성 분석**


*   각 리뷰에 대해 compound 점수를 계산하고, 이를 기준으로 긍정(Positive), 부정(Negative), 중립(Neutral) 감정을 분류.
*   분류 기준 : score > 0.05면 긍정적(Positive), score < 0.05면 부정적(Negative), 그 외는 중립적(Neutral).



In [ ]:
# 1. VADER 감성 분석 수행
# 각 리뷰에 대해 감성 점수(Compound) 계산 및 감정 분류
def get_sentiment(score):
    if score > 0.05:
        return 'Positive'
    elif score < -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# VADER 감성 점수 및 감정 레이블 추가
data['VADER_Score'] = data['Review'].apply(lambda review: analyzer.polarity_scores(review)['compound'])
data['Sentiment'] = data['VADER_Score'].apply(get_sentiment)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**2. 주제별 감정 분석 결과 요약**


*   LDA 토픽 모델링에서 도출된 주제(Topic) 별로 감정 분류 결과를 집계.
*   주제별로 리뷰의 긍정/부정 비율을 계산하여, 각 주제의 감정 분포를 정량적으로 확인.



In [ ]:
# 2. 주제별 감정 분석 결과 요약
# 주제와 감정별로 리뷰 수를 집계
topic_sentiment_summary = data.groupby(['Topic', 'Sentiment']).size().unstack(fill_value=0)

# 각 주제별 리뷰 총합 계산
topic_sentiment_summary['Total'] = topic_sentiment_summary.sum(axis=1)

# 긍정 및 부정 비율 계산
topic_sentiment_summary['Positive_Percentage'] = (topic_sentiment_summary['Positive'] / topic_sentiment_summary['Total']) * 100
topic_sentiment_summary['Negative_Percentage'] = (topic_sentiment_summary['Negative'] / topic_sentiment_summary['Total']) * 100

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**3. 결과 출력**


*   주제별 감정 분포 테이블 출력.
*   각 주제에서 긍정 리뷰 수(Positive), 부정 리뷰 수(Negative), 중립 리뷰 수(Neutral), 긍정/부정 비율(%)이 포함.

**Topic 0 - 호텔 시설 및 환경**

*   **긍정 비율** : 95.97%
*   **부정 비율** : 3.67%
*   **주요 단어** : room, service, area, food, pool, beach, nice, good
*   호텔의 편의시설과 주변환경에 대한 고객의 평가를 반영하였으며, 고객들은 객실, 서비스, 수영장, 해변, 음식에 대해 매우 긍정적인 반응을 보이고 있음.


**Topic 1 - 객실 상태 및 서비스 품질**


*   **긍정 비율** : 68.77%
*   **부정 비율** : 29.51%
*   **주요 단어** : room, staff, service, desk, stay, night, hotel
*   고객들이 객실 상태와 직원 서비스에 대해 평가한 주제로, 부정적 비율이 상대적으로 높아 객실 상태나 서비스 품질에 대한 고객 불만이 존재하고 있음.


**Topic 2 - 직원 친절도와 위치**


*   **긍정 비율** : 99.22%
*   **부정 비율** : 0.66%
*   **주요 단어** : friendly, helpful, location, staff, great, stay, hotel
*   직원의 친절함과 호텔 위치에 대해 평가한 주제로, 고객들이 직원의 서비스 태도와 호텔 위치에 대해 매우 긍정적인 반응을 보이고 있음.


**Topic 3 - 청결도와 아침 식사 품질**


*   **긍정 비율** : 97.10%
*   **부정 비율** : 2.62%
*   **주요 단어** : clean, breakfast, nice, good, location, room
*   객실 청결도와 아침 식사 품질에 대해 평가한 주제로, 긍정적 비율이 높아 전반적으로 고객들이 만족하는 반응을 보이고 있음.


**Topic 4 - 전반적인 고객 경험 및 휴식**


*   **긍정 비율** : 94.28%
*   **부정 비율** : 5.50%
*   **주요 단어** : day, time, people, beach, good, great, room
*   고객들이 호텔에서의 전체적인 경험(휴식, 환경, 서비스)을 다룬 주제로, 주제에서 긍정적인 평가가 우세하며, 고객들이 대체로 호텔에서 편안한 시간을 보냈음을 나타내고 있음.














In [ ]:
# 3. 결과 출력
print("\n주제별 감정 분석 요약:")
print(topic_sentiment_summary)


주제별 감정 분석 요약:
Sentiment  Negative  Neutral  Positive  Total  Positive_Percentage  \
Topic                                                                
0                52        5      1359   1416            95.974576   
1              1051       61      2449   3561            68.772817   
2                36        6      5377   5419            99.224949   
3               179       19      6643   6841            97.105686   
4               179        7      3068   3254            94.283958   

Sentiment  Negative_Percentage  
Topic                           
0                     3.672316  
1                    29.514181  
2                     0.664329  
3                     2.616577  
4                     5.500922  


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
